# オッズの予測精度評価

In [14]:
# Resultモデルをデータフレームに変換
import pandas as pd
from datetime import date
ext_dt = date(2017,1,1)
results = Result.objects.filter(race__race_dt__gte=ext_dt, race__result_flg=True).exclude(rank=0).values("rank", "odor")
df = pd.DataFrame.from_records(results)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59428 entries, 0 to 59427
Data columns (total 2 columns):
odor    59428 non-null int64
rank    59428 non-null int64
dtypes: int64(2)
memory usage: 928.6 KB


In [16]:
import numpy as np

# 目的変数の作成（3位以内ならゼロ、それ以外は1）
df["top3"] = np.where(df["rank"]<=3, 0, 1)
df["odor3"] = np.where(df["odor"]<=3, 0, 1)

# 不要データの削除
df = df.drop("rank", axis=1)
df = df.drop("odor", axis=1)

# データ確認
df.head(10)

,top3,odor3
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [ ]:
from sklearn.model_selection import train_test_split

# 学習データとテストデータの分離
X = df.drop("top3", axis=1)
y = df.top3
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)